<a href="https://colab.research.google.com/github/gimb99/PDH_INTEGRADOR_GRUPAL_X/blob/develop/TI_CTG_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📥 Paso 2.1 – Ingesta de documentos PDF y traducción automática

Este paso carga los documentos técnicos desde el disco local.
Se leen PDFs tanto en español como en inglés.  
Los documentos en inglés se traducen automáticamente al español con un modelo de Hugging Face.
El resultado final es un corpus unificado en español que se utilizará en los siguientes pasos del sistema RAG.


In [ ]:
# 📌 Paso 2.1 – Ingesta y traducción automática de documentos PDF

# ========================
# ✅ Librerías necesarias
# ========================
!pip install --upgrade langchain langchain-community langchain-core langchain-text-splitters transformers pypdf pymupdf  # Upgrade core libraries
!pip install -U huggingface_hub# Install compatible huggingface_hub
from langchain_community.document_loaders import PyMuPDFLoader  # Para cargar PDFs

from transformers import pipeline                                # Para traducir texto
import os                                                        # Para acceder a archivos en disco


!pip install chromadb
from langchain_community.vectorstores import Chroma

In [ ]:
# ========================
# ✅ Modelo de traducción
# ========================
# Este modelo traduce texto de inglés a español usando Hugging Face
translator = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")


In [ ]:
# Paso 2.1 – INGESTA DE DOCUMENTOS: Carga de archivos PDF desde Google Drive

# ======================================
# 🔧 1. Montamos Google Drive (solo en Colab)
# ======================================
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# ======================================
# 📁 2. Definimos las carpetas del corpus
# ======================================
carpetas_es = "/content/drive/MyDrive/ProcHabla/Trabajo Integrador/corpus/corpus_base"

carpeta_en = "/content/drive/MyDrive/ProcHabla/Trabajo Integrador/corpus/complementos_tecnicos"


In [ ]:
# ======================================
# 📚 3. Función para cargar documentos PDF
# ======================================
from langchain_community.document_loaders import PyMuPDFLoader
import os

def cargar_documentos_desde_carpeta(rutas_carpeta):
    documentos = []
    for ruta in rutas_carpeta:
        for archivo in os.listdir(ruta):
            if archivo.endswith(".pdf"):
                ruta_completa = os.path.join(ruta, archivo)
                loader = PyMuPDFLoader(ruta_completa)
                documentos_pdf = loader.load()
                documentos.extend(documentos_pdf)
    return documentos

In [ ]:
# ======================================
# 🌍 4. Función para traducir texto de inglés a español
# ======================================
from transformers import MarianMTModel, MarianTokenizer
import torch

# Modelo para traducir de inglés a español
modelo_trad = "Helsinki-NLP/opus-mt-en-es"
tokenizer_trad = MarianTokenizer.from_pretrained(modelo_trad)
model_trad = MarianMTModel.from_pretrained(modelo_trad)

def traducir_texto(texto, max_length=512):
    oraciones = [texto[i:i+max_length] for i in range(0, len(texto), max_length)]
    resultado = []
    for segmento in oraciones:
        inputs = tokenizer_trad(segmento, return_tensors="pt", truncation=True)
        translated = model_trad.generate(**inputs)
        texto_traducido = tokenizer_trad.decode(translated[0], skip_special_tokens=True)
        resultado.append(texto_traducido)
    return " ".join(resultado)


In [ ]:
# ======================================
# 📄 5. Traducción de documentos en inglés
# ======================================
def traducir_documentos_en_ingles(ruta):
    documentos = []
    for archivo in os.listdir(ruta):
        if archivo.endswith(".pdf"):
            ruta_completa = os.path.join(ruta, archivo)
            loader = PyMuPDFLoader(ruta_completa)
            docs = loader.load()
            for doc in docs:
                texto_original = doc.page_content
                texto_traducido = traducir_texto(texto_original)
                doc.page_content = texto_traducido
                documentos.append(doc)
    return documentos


In [ ]:
# ======================================
# 📦 6. Ejecutamos la carga total del corpus
# ======================================
documentos_es = cargar_documentos_desde_carpeta([carpetas_es])
documentos_en = traducir_documentos_en_ingles(carpeta_en)

documentos = documentos_es + documentos_en

CHUNKING (División en fragmentos)

In [ ]:
# 📌 VALIDACIONES previas antes del chunking

print("🔎 Validando el corpus final...")

# 1. ¿Qué tipo de objeto es el corpus?
print(f"Tipo de corpus_completo: {type(documentos)}")

# 2. ¿Cuántos documentos contiene?
print(f"📚 Total de documentos: {len(documentos)}")

# 3. ¿Qué tipo de objeto es cada documento?
if documentos:
    print(f"Ejemplo de tipo de documento: {type(documentos[0])}")

# 4. Mostrar los primeros 500 caracteres del primer documento
if documentos and hasattr(documentos[0], "page_content"):
    print("\n📝 Vista previa del primer documento:")
    print(documentos[0].page_content[:500])
else:
    print("⚠️ No se encontró texto en el primer documento.")


CHUNKING (División en fragmentos)

In [ ]:
# ✅ Importamos la herramienta para dividir documentos en fragmentos/chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# ✅ Configuramos el splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Cantidad máxima de caracteres por fragmento
    chunk_overlap=50,      # Cuántos caracteres se solapan entre fragmentos
    separators=["\n\n", "\n", ".", " ", ""],  # Orden de preferencia para cortar texto
)


In [ ]:
# ✅ Aplicar el splitter a todos los documentos
chunks = text_splitter.split_documents(documentos)


In [ ]:
from pathlib import Path

# ✅ Enriquecer y simplificar la metadata de cada chunk
for chunk in chunks:
    metadata = chunk.metadata

    # Extraer nombre de archivo limpio
    file_name = Path(metadata.get("file_path", "desconocido")).stem

    # Agregar nombre simple
    metadata["nombre_documento"] = file_name

    # Agregar categoría manual según carpeta origen
    if "corpus_base" in metadata.get("file_path", ""):
        metadata["categoria"] = "base_tecnica"
        metadata["idioma"] = "es"
    elif "complementos_tecnicos" in metadata.get("file_path", ""):
        metadata["categoria"] = "complemento"
        metadata["idioma"] = "en"
    else:
        metadata["categoria"] = "otro"
        metadata["idioma"] = "desconocido"

    # Limpiar metadata innecesaria
    for campo in ["producer", "creator", "format", "encryption", "trapped", "moddate", "creationdate", "title", "author", "subject", "keywords"]:
        metadata.pop(campo, None)


In [ ]:
# Validar resultados del chunking
print(f"📄 Total de chunks generados: {len(chunks)}")
print("\n📝 Primer chunk:")
print(chunks[0].page_content[:500])


In [ ]:
# Revisar la metadata de los primeros 3 chunks
for i, chunk in enumerate(chunks[:3]):
    print(f"\n🧾 Chunk {i+1} - Metadata:")
    print(chunk.metadata)


EMBEDDINGS

In [ ]:
# ==========================
# Paso 3.1: Cargar modelo de embeddings en español
# ==========================

from langchain_huggingface import HuggingFaceEmbeddings # Updated import

# Definir el modelo de embeddings
modelo_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

In [ ]:

# ===============================================
# 🧠 Paso 3.2: Vectorización de Chunks con ChromaDB
# ===============================================

# Ruta donde se almacenará la base vectorial
persistencia_vectores = "db_vectores"

# Crear base de datos vectorial con los embeddings
chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=modelo_embeddings,
    persist_directory="db_vectores"
)

# Guardar la base persistente en disco
chroma_db.persist()

# Validación visual
print("✅ Base de datos vectorial creada con éxito y guardada en:", persistencia_vectores)



In [ ]:
# ====================================================
# 🧠 Paso 5.1: Cargar base vectorial persistente y preparar el Retriever
# ====================================================

from langchain_community.vectorstores import Chroma

# 🔄 Ruta donde guardaste la base de datos vectorial
persistencia_vectores = "db_vectores"

# 🗃️ Cargar la base vectorial persistente desde el disco
chroma_db = Chroma(
    persist_directory=persistencia_vectores,
    embedding_function=modelo_embeddings
)

# 🔍 Crear un Retriever para realizar búsquedas por similitud
retriever = chroma_db.as_retriever(search_kwargs={"k": 3})

# ✅ Validación
print("✅ Retriever creado correctamente. Listo para recuperar chunks similares.")


In [ ]:
# =============================================
# Ƈ Paso 5.2: Consulta de Prueba y Recuperación
# =============================================

# ✅ Imports necesarios para este paso
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document


# Ƈ Cargamos la base vectorial persistida
chroma_db = Chroma(
    persist_directory="db_vectores",
    embedding_function=modelo_embeddings
)

# Ƈ Creamos el retriever (mecanismo de recuperación)
retriever = chroma_db.as_retriever(
    search_type="similarity",  # También puedes usar "mmr" (Maximal Marginal Relevance)
    search_kwargs={"k": 3}      # Número de documentos más similares que queremos recuperar
)

# Ƈ Definimos una pregunta de prueba
pregunta_prueba = "¿Qué es un reservorio no convencional?"

# Ƈ Recuperamos los documentos más relevantes
resultados = retriever.get_relevant_documents(pregunta_prueba)

# Ƈ Mostramos los resultados
print("ƒ Resultados de la recuperación:\n")
for i, doc in enumerate(resultados, 1):
    print(f"ƒ Documento {i}:")
    print(doc.page_content[:500])  # Muestra los primeros 500 caracteres
    print("ƒ Metadata:", doc.metadata)
    print("-" * 80)

In [ ]:
# Paso previo: cargar el modelo de lenguaje LLM desde Hugging Face
from langchain.llms import HuggingFaceHub
import os
from getpass import getpass

# ⚠️ Agrega tu token personal de Hugging Face
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("🔑 Ingresa tu token de Hugging Face: ")

# ✅ Carga del modelo (puedes cambiar por otro compatible)
llm_model = HuggingFaceHub(
    repo_id="google/flan-t5-base",  # Otro modelo posible: "mistralai/Mistral-7B-Instruct-v0.1"
    model_kwargs={"temperature": 0.1, "max_length": 512}
)

# ✅ Ahora sí puedes crear tu cadena RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_model,
    retriever=retriever,
    return_source_documents=True,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_personalizado}
)
